In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/graph-data/new_bus_transport_graph_with_service_no_new.json
/kaggle/input/graph-data/new_bus_transport_graph_with_service_no.json


In [16]:
!pip3 install neo4j
!pip install networkx-neo4j
!pip3 install streamlit_folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 683.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 10.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 20.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=f61f835cc1220379e008270ccd568f88a8c4471559858357e4923c88502516d4
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [27]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import json
import nxneo4j as nxn4j
import folium
from folium import Marker
from folium.plugins import MarkerCluster, HeatMap
from streamlit_folium import st_folium, folium_static
import osmnx as ox

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
def load_graph_nodes(fname):
    d = read_json(fname)
    nodes_info = {}
    for nodes in d['nodes']:
        nodes_info[nodes['id']]=nodes
    return nodes_info

def read_json(file_path):
    f = open(file_path)
    data = json.load(f)
    f.close()
    
    return data

def write_to_json(dict_obj,file_name):
    with open(file_name, "w") as outfile:
        json.dump(dict_obj, outfile)
    print('written file to '+file_name)
    
def save_graph_data(file_name,graph):
    data = json_graph.node_link_data(graph)
    with open(file_name, "w") as outfile:
        json.dump(data, outfile)
    print('written file to '+file_name)

In [ ]:
class App:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()
        
    def create_nodes(self,node_type,source_node):
         with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.execute_write(
                self._create_bus_nodes,node_type,source_node)

    def create_relationships(self,link_type,source_node,target_node,weight,links):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.execute_write(
                self._create_and_return_rs,link_type,source_node,target_node,weight,links)

    @staticmethod
    def _create_bus_nodes(tx,node_type,node):
        query = (
        """
        CREATE (n:bus_node {busstop_id:$source_id,latitude:$lat,longitude:$long,
                            planningArea:$planArea,tap_in:$tap_in,tap_out:$tap_out,
                            deficit:$deficit,avg_flow:$avg_flow,pop_multiplier:$pop_multiplier,
                            pop_estimate:$pop_estimate})
        """)
        result = tx.run(query,node_type=node_type,
                        planArea = node['planningArea'],
                        tap_in = node['tap_in'],
                        tap_out = node['tap_out'],
                        deficit = node['deficit'],
                        avg_flow = node["avg_flow"],
                        pop_multiplier = node["pop_multiplier"],
                        pop_estimate = node["pop_estimate"],
                        lat=node['latitude'],
                        long=node['longitude'],
                        source_id = str(node['id']))
        
    @staticmethod
    def _create_and_return_rs(tx, link_type, source_node, target_node, weight,links):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
        """
        MATCH (n1),(n2)
        WHERE n1.busstop_id = $busstop1 AND n2.busstop_id = $busstop2
        CREATE (n1)-[:bus_bus {weight: $weight,name:$link_type,weekends_trips:$weekends_trips,
                                weekdays_trips:$weekdays_trips,AM_Offpeak_Freq:$AM_Offpeak_Freq,
                                AM_Peak_Freq:$AM_Peak_Freq,PM_Offpeak_Freq:$PM_Offpeak_Freq,
                                PM_Peak_Freq:$PM_Peak_Freq,service_list:$service_list}]->(n2)
        return n1,n2
        """)
        
        result = tx.run(query,link_type=link_type,
                        busstop1=source_node['id'],busstop2=target_node['id'],
                        weekends_trips = links["weekends_trips"],
                        weekdays_trips = links["weekdays_trips"],
                        AM_Offpeak_Freq = links["AM_Offpeak_Freq"],
                        AM_Peak_Freq = links["AM_Peak_Freq"],
                        PM_Offpeak_Freq = links["PM_Offpeak_Freq"],
                        PM_Peak_Freq = links["PM_Peak_Freq"],
                        service_list = (links["service_list"]),
                        weight=weight)
        
    def find_person(self, bus_stop):
        with self.driver.session(database="neo4j") as session:
            result = session.execute_read(self._find_and_return_person, bus_stop)
            for row in result:
                print("hello")
                return False
        return True

    def find_person_static(bus_stop):
        with self.driver.session(database="neo4j") as session:
            result = session.execute_read(_find_and_return_person, bus_stop)
            for row in result:
                print("hello")
                return False
        return True
    
    def _find_and_return_person(tx, bus_stop):
        query = (
            "MATCH (n) "
            "WHERE n.busstop_id = $bus_stop "
            "RETURN n"
        )
        result = tx.run(query, bus_stop=bus_stop)
        return [row for row in result]


if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
#     uri      = "neo4j+s://2c5e8457.databases.neo4j.io:7687" # in Neo4j Desktop
#     uri      = "neo4j+s://2c5e8457.databases.neo4j.io"
#     # uri      = "bolt://localhost:7687"
#                                   # custom URL for Sandbox or Aura
#     user     = "neo4j"            # your user name 
#                                   # default is always "neo4j" 
#                                   # unless you have changed it. 
#     password = 'C3DKuamyGb64q9f8ldtq4u0N9Jjxag-9eRPceC0MwPQ'

    uri='neo4j+s://7be14e4d.databases.neo4j.io'
    user='neo4j'
    password='Wm9lnnu0db4fD_g9IOAf67zKNk8O6mCrpgk7lq2j3uI'
    
    fname = '/kaggle/input/graph-data/new_bus_transport_graph_with_service_no_new.json'
    app = App(uri, user, password)
    nodes_info = load_graph_nodes(fname)
    d = read_json(fname)
    nodes_list = []
    nodes_list = list(set(nodes_list))
    for n in nodes_info:
        app.create_nodes('busstop',nodes_info[n])
    print('uploaded nodes')
    for links in d['links']:
        source_node = nodes_info[links['source']]
        target_node = nodes_info[links['target']]
        weight = links['weight']
        app.create_relationships('bus_bus',source_node,target_node,weight,links)
    print('uploaded relationships')
    app.close()

In [5]:
uri='neo4j+s://7be14e4d.databases.neo4j.io'
user='neo4j'
password='Wm9lnnu0db4fD_g9IOAf67zKNk8O6mCrpgk7lq2j3uI'
driver = GraphDatabase.driver(uri=uri,auth=(user,password))
# transport_graph = nxn4j.DiGraph(driver) # directed graph
# transport_graph.delete_all()  #This will delete all the data, be careful

In [7]:
# Run cql and return neo4j result
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return result.data()

# Query returns all unique planning area
def get_planning_areas():
    query = """
    MATCH (n) 
    RETURN DISTINCT n.planningArea
    """
    result = run_query(query)
    res = [i['n.planningArea'] for i in result]
    return res

# Query get nodes in planning area
def query_planningArea(planningArea):
    query = ("""
    MATCH (n1),(n2) MATCH (n1)-[r]-(n2) 
    WHERE n1.planningArea = '"""+planningArea+"""' OR n2.planningArea = '"""+planningArea+"""'
    RETURN n1.latitude,n1.longitude, n2.latitude,n2.longitude
    """)
    result = run_query(query)
    res = [((i['n1.latitude'], i['n1.longitude']), (i['n2.latitude'], i['n2.longitude'])) for i in result]
    return res

In [25]:
planning_areas = get_planning_areas()
filter_links = query_planningArea('DOWNTOWN CORE')

In [42]:
base=folium.Map([1.36, 103.83],zoom_start=11.5,tiles="cartodbpositron")
for i in range(len(filter_links)):
    # for pos_lat_long in bus_stops:
    #     # print("printing:",pos_lat_long)
    folium.PolyLine(
            filter_links[i], # tuple of coordinates 
            color = 'blue', # map each segment with the speed 
            # colormap = color_dict, # map each value with a color 
            ).add_to(base)
        # print(pos_lat_long)
base
# st_folium(base, width=1200, height=600)